In [1]:
# pip install google-cloud-vision

# 로컬 이미지로 텍스트 추출

import os
import re
import json
import requests
import pandas as pd
from io import BytesIO
from PIL import Image
from google.cloud import vision

# import ray
# from pykospacing import Spacing
# from konlpy.tag import Kkma, Okt
# from nltk.tokenize import word_tokenize


# 개인 api 키(개인적으로 발급 받아야 됨)
# 키 발급 받는 방법
# https://ssungkang.tistory.com/entry/Google-Vision-API-%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%9C-%EA%B8%80%EC%9E%90-%EC%9D%B8%EC%8B%9D

def image_text(category):
    result = []
    count = 0
    # 개인 api 키(개인적으로 발급 받아야 됨)
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\gcp_jason\123456789.json"    ##
    
    folder_path1 = f"C:/wadiz/{category}"    ##
    code_result = os.listdir(folder_path1)
    # code_result = code_result[:1000]    # 1000개 까지만 변환
    
    for i in code_result:
        text_result = ''
        folder_path = f"C:/wadiz/{category}/{i}"    ##
        file_names = os.listdir(folder_path)
        
        for k in file_names:
            try :
                if "gif" in k:
                    continue

                # 이미지 파일 열기
                image_data = Image.open(f"C:/wadiz/{category}/{i}/{k}")    ##
                
                # 이미지를 bytes로 변환
                if "png" in k:
                    image_bytes = BytesIO()
                    image_data.save(image_bytes, format='PNG')
                    image_data = image_bytes.getvalue()
                elif 'jpg' in k:
                    image_bytes = BytesIO()
                    image_data.save(image_bytes, format='JPEG')
                    image_data = image_bytes.getvalue()

                # Vision API 클라이언트 생성
                client = vision.ImageAnnotatorClient()

                # Vision API에 전달할 이미지 생성
                image = vision.Image(content=image_data)

                # 이미지에서 텍스트 감지
                res = client.text_detection(image=image)
                text_annotations = res.text_annotations

                if text_annotations != []:
                    text = text_annotations[0].description
                    text_result = text_result + text
            except:
                pass

        print(f"{count}/{len(code_result)} 완료")
        result.append([i, text_result])
        count += 1

    df = pd.DataFrame(result, columns=['code', 'img_text'])
    df.to_csv(f"이미지 텍스트 추출_{category}.csv")

image_text(287) # 카테고리 번호